## INTROTONEURALNETWORKS/3 INTROTOTENSORFLOW/INTROTONEURALNETWORKS INTROTOTENSORFLOW 2 EXERCISE ANSWERS ##
#### Exercise ####
#### Please refer to module 2 of IntroToNeuralNetworks - IntroToTensorflow for Tasks 1-7
#### Task 1 
##### Load the libraries that are used in this module.
#### Result:


In [ ]:
# Helper packages.
import os
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import seaborn as sns

# Scikit-learn packages.
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics

# TensorFlow and supporting packages.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

#### Task 2
##### Define the data directory.
#### Result:


In [ ]:
from pathlib import Path

home_dir = Path(".").resolve()
main_dir = home_dir.parent.parent
print(main_dir)
data_dir = str(main_dir) + "/data"
print(data_dir)

#### Task 3
##### Load the dataset `bank_marketing.csv` and save it to `bank_marketing`.
##### Print the first few rows of `bank_marketing`.
#### Result:


In [ ]:
bank_marketing = pd.read_csv(data_dir + "/bank_marketing.csv")
bank_marketing.head()

#### Task 4
##### Define a convenience function `ex_data_prep` to perform the data cleaning steps mentioned below.


In [ ]:
1. Replace the column `y` in the dataframe, by setting it to 1 if `y` is 'yes', otherwise set `y` to 0.
2. Perform one hot encoding on the variables with data type object (i.e `job`, `marital`, `education`, `default`, `housing`, `loan`, `contact`, `month`, `day_of_week` and `poutcome`) except the target variable `y`
3. Drop the original variables and concatenate the dummies to original datset
4. Select the predictors by dropping variable `y` and save the result to a dataframe `X_ex`
5. Save the target variable `y` column to `y_ex` variable
6. Set the seed to 1
7. Split the data into training, test, and validation sets with 70:15:15 ratio and save respective variables to `X_train_ex`, `X_test_ex`, `X_val_ex`, `y_train_ex`, `y_test_ex`, `y_val_ex`
8. Scale the train, test and the validation datasets using Min max scaler and save as `X_train_scaled_ex`, `X_test_scaled_ex` and `X_val_scaled_ex` respectiely


#### Result:


In [ ]:
def ex_data_prep(df):

    # Convert `y` to 0/1 values
    df["y"] = np.where(df["y"] == "yes", 1, 0)

    # Perform one hot encoding
    job_dummy = pd.get_dummies(df["job"], prefix="job", drop_first=True)
    marital_dummy = pd.get_dummies(df["marital"], prefix="marital", drop_first=True)
    education_dummy = pd.get_dummies(
        df["education"], prefix="education", drop_first=True
    )
    default_dummy = pd.get_dummies(df["default"], prefix="default", drop_first=True)
    housing_dummy = pd.get_dummies(df["housing"], prefix="housing", drop_first=True)
    loan_dummy = pd.get_dummies(df["loan"], prefix="loan", drop_first=True)
    contact_dummy = pd.get_dummies(df["contact"], prefix="contact", drop_first=True)
    month_dummy = pd.get_dummies(df["month"], prefix="month", drop_first=True)
    day_of_week_dummy = pd.get_dummies(
        df["day_of_week"], prefix="day_of_week", drop_first=True
    )
    poutcome_dummy = pd.get_dummies(df["poutcome"], prefix="poutcome", drop_first=True)

    # Drop the original variables
    df.drop(
        [
            "job",
            "marital",
            "education",
            "default",
            "housing",
            "loan",
            "contact",
            "month",
            "day_of_week",
            "poutcome",
        ],
        axis=1,
        inplace=True,
    )

    # Concatenate the dummies to original dataset
    df = pd.concat(
        [
            df,
            job_dummy,
            marital_dummy,
            education_dummy,
            default_dummy,
            housing_dummy,
            loan_dummy,
            contact_dummy,
            month_dummy,
            day_of_week_dummy,
            poutcome_dummy,
        ],
        axis=1,
    )

    # Separate predictors from target variable.
    X_ex = df.drop(["y"], axis=1)
    y_ex = df["y"]

    # Set the seed to 1.
    np.random.seed(1)
    # Split data into train, test, and validation set, use a 70 - 15 - 15 split.
    # First split data into train-test with 70% for train and 30% for test.
    X_train_ex, X_test_ex, y_train_ex, y_test_ex = train_test_split(
        X_ex.values, y_ex, test_size=0.3, random_state=1
    )
    # Then split the test data into two halves: test and validation.
    X_test_ex, X_val_ex, y_test_ex, y_val_ex = train_test_split(
        X_test_ex, y_test_ex, test_size=0.5, random_state=1
    )
    print(
        "Train shape:",
        X_train_ex.shape,
        "Test shape:",
        X_test_ex.shape,
        "Val shape:",
        X_val_ex.shape,
    )

    # Transforms features by scaling each feature to a given range.
    # The default is the range between 0 and 1.
    min_max_scaler = preprocessing.MinMaxScaler()
    X_train_scaled_ex = min_max_scaler.fit_transform(X_train_ex)
    X_test_scaled_ex = min_max_scaler.transform(X_test_ex)
    X_val_scaled_ex = min_max_scaler.transform(X_val_ex)

    return (
        X_train_scaled_ex,
        X_test_scaled_ex,
        X_val_scaled_ex,
        y_train_ex,
        y_test_ex,
        y_val_ex,
    )


(
    X_train_scaled_ex,
    X_test_scaled_ex,
    X_val_scaled_ex,
    y_train_ex,
    y_test_ex,
    y_val_ex,
) = ex_data_prep(bank_marketing)

#### Task 5
##### Initialize a simple sequential neural network model with 32 neurons for the 1st hidden layer, 20 neurons for the second layer, and appropriate input and output layers. Name the model `model_ex`.
##### Compile the model using the "adam" optimizer and "binary_crossentropy" loss, and use "accuracy" as a metric.
##### Fit the model using train and validation sets with 200 epochs, and assign it to `model_res_ex` variable.
#### Result:


In [ ]:
# Let's set the seed so that we can reproduce the results.
tf.random.set_seed(1)

## Set up model.


In [ ]:
model_ex = Sequential(
    [
        Dense(
            32,  # <- number of neurons for 1st hidden layer
            input_shape=(X_train_scaled_ex.shape[1],),
        ),  # <- input layer shape: `(num_features, )`
        Dense(
            20, activation="relu"  # <- add 2nd hidden layer with 20 neurons
        ),  # <- set activation function for hidden layer
        Dense(
            1, activation="sigmoid"  # <- add output layer with single neuron
        ),  # <- activation function for output layer
    ]
)
# Compile the model.
model_ex.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Fitting the model
model_res_ex = model_ex.fit(
    X_train_scaled_ex,
    y_train_ex,  # <- train data and labels
    validation_data=(X_val_scaled_ex, y_val_ex),  # <- pass validation data
    epochs=200,
)  # <- specify number of epochs

#### Task 6
##### Plot accuracy and loss curves for `model_res_ex`. 
##### You can access them by checking the `model_res_ex.history` dictionary.
#### Result:


In [ ]:
# Plot training & validation accuracy values
plt.plot(model_res_ex.history["accuracy"])  # <- accuracy scores
plt.plot(
    model_res_ex.history["val_accuracy"]
)  # <- get validation accuracy scores from dictionary
plt.title("Model accuracy")  # <- adding title to the plot
plt.ylabel("Accuracy")  # <- naming the y-axis
plt.xlabel("Epoch")  # <- naming the x-axis
plt.legend(["Train", "Val"], loc="upper left")  # <- adding a legend
plt.show()  # <- displaying the plot
# Plot training & validation loss values
plt.plot(model_res_ex.history["loss"])  # <- loss values
plt.plot(
    model_res_ex.history["val_loss"]
)  # <- get validation loss scores from dictionary
plt.title("Model loss")  # <- adding title to the plot
plt.ylabel("Loss")  # <- naming the y-axis
plt.xlabel("Epoch")  # <- naming the x-axis
plt.legend(["Train", "Val"], loc="upper left")  # <- adding a legend
plt.show()  # <- displaying the plot

#### Task 7
##### Evaluate the model on the test data. Save loss as `loss_ex` and accuracy as `accuracy_ex`. Print the loss and accuracy values.
##### Predict the labels on test data for the model and save as `y_pred_ex`. 
##### Check how the values look.
#### Result:


In [ ]:
loss_ex, accuracy_ex = model_ex.evaluate(x=X_test_scaled_ex, y=y_test_ex)
print("Loss: {0:6.3f}, Accuracy: {1:6.3f}".format(loss_ex, accuracy_ex))
y_pred_ex = (model_ex.predict(X_test_scaled_ex) > 0.5).astype("int32")
print(y_pred_ex)